This is a assignment of COMP3361 in HKU

https://nlp.cs.hku.hk/comp3361/hw1.pdf

# 0. Install the requied package


In [ ]:
!nvidia-smi

Mon Sep 20 04:38:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
! pip install datasets transformers

     |████████████████████████████████| 270 kB 5.1 MB/s 
     |████████████████████████████████| 2.8 MB 69.8 MB/s 
     |████████████████████████████████| 52 kB 2.0 MB/s 
     |████████████████████████████████| 243 kB 79.3 MB/s 
     |████████████████████████████████| 119 kB 82.9 MB/s 
     |████████████████████████████████| 1.3 MB 55.1 MB/s 
     |████████████████████████████████| 895 kB 71.1 MB/s 
     |████████████████████████████████| 636 kB 86.4 MB/s 
     |████████████████████████████████| 3.3 MB 76.4 MB/s 
     |████████████████████████████████| 142 kB 69.8 MB/s 
     |████████████████████████████████| 294 kB 62.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        
Username: cogito@connect.hku.hk
Password: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-crendential store but this isn't the helper defined on your machine.
You will have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal to set it as the default

git config --global credential.helper store


In [ ]:
!apt install git-lfs
!git config --global user.email "cogito@connect.hku.hk"
!git config --global user.name "cogito233"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 37 not upgraded.
Need to get 2,129 kB of archives.
After this operation, 7,662 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 git-lfs amd64 2.3.4-1 [2,129 kB]
Fetched 2,129 kB in 1s (1,520 kB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 155013 files and directories currently installed.)
Preparing to unpack .../git-lfs_2.3.4-1_amd64.deb ...
Unpacking git-lfs (2.3.4-1) ...
Setting up git-lfs (2.3.4-1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...


# 1. Parpare the data
We use the codabase `transformers` and its `datasets` to preprocess the data.

Firstly, we try to observe the data.

In [ ]:
DATA_PATH = '/content/drive/MyDrive/COMP3361/Assignment1/data/'
!head -n 10 /content/drive/MyDrive/COMP3361/Assignment1/data/valid.fo1

homarus gammarus , known as the european lobster or common lobster , is a species **GW** <unk> lobster from the eastern atlantic ocean , mediterranean sea and parts **GW** the black sea . it is closely related to the american lobster , h. americanus . it may grow to a length **GW** 60 cm ( 24 **GW** ) and a mass **GW** 6 kilograms ( 13 lb ) , and bears a conspicuous pair **GW** claws . **GW** life , the lobsters are blue , only becoming " lobster red " **GW** cooking . mating occurs **GW** the summer , producing eggs which are carried by the females **GW** up to a year before hatching into <unk> larvae . homarus gammarus is a highly esteemed food , and is widely caught using lobster pots , mostly around the british isles .
homarus gammarus is a large <unk> , with a body length up to 60 centimetres ( 24 **GW** ) and weighing up to 5 – 6 kilograms ( 11 – 13 lb ) , although the lobsters caught **GW** lobster pots are usually 23 – 38 cm ( 9 – 15 **GW** ) long and weigh 0 @.@ 7 – 2 @.@ 2 kg

In [ ]:
!head -10 /content/drive/MyDrive/COMP3361/Assignment1/data/valid.fo2

of,of,of,in,of,of,in,on,in,for
in,in,in,in,in,for,for
of,of,for,for
in,of
of,in,of
of,on,in
on,of,of
of,of,of
of,in,at,in,for,on
at,on,for,on,in,of,in,of,in


## 1. The way to filter the data
By observe the data, we find the data is composed by some different short sentence, seperated by "," or ".".

So firstly, we seperate the data by "," and ".", and then discard the data which is not include the “at, in, of, for on”.


In [ ]:
class DataFliter(object):
    def __init__(self, data_path, data_name):
        self.data = []
        ans = 0
        with open(data_path+data_name, 'r') as f:
            for line in f:
                for sentence in line.split('.'):
                    for subsentence in sentence.split(';'):
                        if len(subsentence)>300:
                            for subsubsentence in subsentence.split(','):
                                if (self.check_valid(subsubsentence)):
                                    self.data.append(subsubsentence)
                        else:
                            if (self.check_valid(subsentence)):
                                self.data.append(subsentence)
    def check_valid(self, s):
        return (' at ' in s) or (' in ' in s) or (' of ' in s) or (' for ' in s) or (' on ' in s) or ('**GW**' in s)
    def __getitem__(self, idx):
        return self.data[idx]
    def __len__(self):
        return len(self.data)

In [ ]:
class DataFliter_subsentence(object):
    def __init__(self, data_path, data_name):
        self.data = []
        ans = 0
        with open(data_path+data_name, 'r') as f:
            for line in f:
                for sentence in line.split('.'):
                    for subsentence in sentence.split(';'):
                        for subsubsentence in subsentence.split(','):
                            if subsubsentence[len(subsubsentence)-1]=='\n':
                                subsubsentence=subsubsentence[0:len(subsubsentence)-1]
                            if subsubsentence!='':
                                self.data.append(subsubsentence)
    def check_valid(self, s):
        return ('at' in s) or (' in ' in s) or (' of ' in s) or (' for ' in s) or (' on ' in s) or (' **GW** ' in s)
    def __getitem__(self, idx):
        return self.data[idx]
    def __len__(self):
        return len(self.data)

In [ ]:
train_data = DataFliter(DATA_PATH, 'train.fo1')

In [ ]:
print(train_data[0])
print(train_data[1])
print(train_data[2])

 valkyria of the battlefield 3 ) , commonly referred to as valkyria chronicles iii outside japan , is a tactical role @-@ playing video game developed by sega and media
vision for the playstation portable 
 released in january 2011 in japan , it is the third game in the valkyria series 


## 2. Use BPE tokenizer
https://huggingface.co/docs/tokenizers/python/latest/quicktour.html


In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

Found existing installation: tensorflow 2.6.0
Uninstalling tensorflow-2.6.0:
  Successfully uninstalled tensorflow-2.6.0
  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-nfvm9exh
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-nfvm9exh
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.11.0.dev0-py3-none-any.whl size=2833295 sha256=243d02b9a775cc97e494fd9943111ddcbc48f4bfc5f703cdf040b3aa422f015b
  Stored in directory: /tmp/pip-ephem-wheel-cache-a8vyvqxm/wheels/35/2e/a7/d819e3310040329f0f47e57c9e3e7a7338aa5e74c49acfe522
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.10.2
    Uninstalling transformers-4.10.2:
      Successfully uninstalled transformers-4.10.2
tokenizers                    0.10.3
transformers         

In [ ]:
from pathlib import Path

from tokenizers import ByteLevelBPETokenizer

files = ["/content/drive/MyDrive/COMP3361/Assignment1/data/valid.fo1",
         "/content/drive/MyDrive/COMP3361/Assignment1/data/train.fo1"]
paths = files

# Initialize a tokenizer
tokenizer = ByteLevelBPETokenizer()

# Customize training
tokenizer.train(files=paths, vocab_size=10000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
    "**GW**",
])

In [ ]:
!mkdir tokenizer
tokenizer.save_model("./tokenizer")

mkdir: cannot create directory ‘tokenizer’: File exists


['./tokenizer/vocab.json', './tokenizer/merges.txt']

In [ ]:
x=tokenizer.encode(" **GW** in forest in **GW** 2011 in japan , it is the third game in the valkyria series ")

#### 1. 上面是标准模板捏

In [ ]:
from tokenizers import Tokenizer
from tokenizers.models import BPE

tokenizer = Tokenizer(BPE(unk_token="[UNK]"))

from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(vocab_size=10000,special_tokens=["[UNK]", "[CLS]", "[SEP]",
                                "[PAD]", "[MASK]","**GW**","at", 'in', 'of', 'for', 'on'])


from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
tokenizer.pre_tokenizer = Whitespace()


files = ["/content/drive/MyDrive/COMP3361/Assignment1/data/valid.fo1",
         "/content/drive/MyDrive/COMP3361/Assignment1/data/train.fo1"]

tokenizer.train(files, trainer)

tokenizer.save("tokenizer-wiki.json")

tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

In [ ]:
tokenizer.post_processor = TemplateProcessing(
    single="[CLS] $A [SEP]",
    pair="[CLS] $A [SEP] $B:1 [SEP]:1",
    special_tokens=[
        ("[CLS]", tokenizer.token_to_id("[CLS]")),
        ("[SEP]", tokenizer.token_to_id("[SEP]")),
    ],
)

In [ ]:
tokenizer = Tokenizer.from_file("tokenizer-wiki.json")

In [ ]:
x.tokens

['Ġ',
 '**GW**',
 'Ġin',
 'Ġforest',
 'Ġin',
 'Ġ',
 '**GW**',
 'Ġ2011',
 'Ġin',
 'Ġjapan',
 'Ġ,',
 'Ġit',
 'Ġis',
 'Ġthe',
 'Ġthird',
 'Ġgame',
 'Ġin',
 'Ġthe',
 'Ġvalkyria',
 'Ġseries',
 'Ġ']

In [ ]:
def tokenizerFunc(str):
    x = tokenizer.encode(str)
    return {"input_ids":x.ids, "attention_mask":x.attention_mask}

## 3. Mask the data

In [ ]:
class DataGenerator(object):
    '''使用pandas来完成数据提取（复杂度有点感人，但只能选择相信pandas内置函数的速度了）'''
    def __init__(self, tokenizer):
        # 分词器
        self.tokenizer = tokenizer
        self.label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on', 'Ġat', 'Ġin', 'Ġof', 'Ġfor', 'Ġon']
        self.label_dict = {'**GW**':0, 'at':1, 'in':2, 'of':3, 'for':4, 'on':5,
                           'Ġat':1, 'Ġin':2, 'Ġof':3, 'Ġfor':4, 'Ġon':5
                           }
        self.label_stat = [0,0,0,0,0,0]

    def tokenizerFunc(self,str):
        x = self.tokenizer.encode(str)
        return {"input_ids":x.ids, "tokens":x.tokens, "attention_mask":x.attention_mask}

    def maskFunc(self, str):
        data_dict = self.tokenizerFunc(str)
        n = len(data_dict["input_ids"])
        if (n>128):
            print(n)
        data_dict["label"] = [-100]*n
        for i in range(n):
            if data_dict["tokens"][i] in self.label_list:
                data_dict["attention_mask"][i] = 0
                data_dict["label"][i] = self.label_dict[data_dict["tokens"][i]]
                self.label_stat[self.label_dict[data_dict["tokens"][i]]] += 1
        data_dict['input_ids'] += [0]*(128-n)
        data_dict['tokens'] += ['']*(128-n)
        data_dict['attention_mask'] += [0]*(128-n)
        data_dict['label'] += [-100]*(128-n)
        return data_dict

    def __call__(self, example):
        return self.maskFunc(example)



In [ ]:
dataGenerator = DataGenerator(tokenizer)
print(dataGenerator(" released in january 2011 in japan , it is the third game in the valkyria series ")["attention_mask"])

AttributeError: ignored

## 4. Generate the Train and Test set

In [ ]:
dataGenerator = DataGenerator(tokenizer)
dict_data={'input_ids':[]}
for i in train_data:
    example = dataGenerator(i)
    dict_data['input_ids'].append(example['input_ids'])

from datasets import Dataset
dataset = Dataset.from_dict(dict_data)

In [ ]:
dataGenerator = DataGenerator(tokenizer)
dict_data={'input_ids':[], 'tokens':[], 'attention_mask':[], 'label':[]}
for i in train_data:
    example = dataGenerator(i)
    dict_data['input_ids'].append(example['input_ids'])
    dict_data['tokens'].append(example['tokens'])
    dict_data['attention_mask'].append(example['attention_mask'])
    dict_data['label'].append(example['label'])

from datasets import Dataset
train_dataset = Dataset.from_dict(dict_data)

In [ ]:
dataset

Dataset({
    features: ['input_ids', 'tokens', 'attention_mask', 'label'],
    num_rows: 66828
})

In [ ]:
import datasets
dataset = train_dataset
dataset.shuffle()
sub_dataset = [dataset.shard(num_shards =5 , index = i) for i in range(5)]
train_set = datasets.concatenate_datasets([sub_dataset[j] for j in range(5) if j!=0])
text_set = sub_dataset[0]

In [ ]:
text_set

Dataset({
    features: ['input_ids', 'tokens', 'attention_mask', 'label'],
    num_rows: 13366
})

## 4.1. Loading the test datasets

In this section, we load the labels in valid.fo2 to the valid set


In [ ]:
valid_data = DataFliter(DATA_PATH, 'valid.fo1')
valid_data_label = DataFliter_subsentence(DATA_PATH,'valid.fo2')

In [ ]:
len(valid_data)

7057

In [ ]:
class DataGenerator_valid(object):
    '''使用pandas来完成数据提取（复杂度有点感人，但只能选择相信pandas内置函数的速度了）'''
    def __init__(self, tokenizer, labels):
        # 分词器
        self.tokenizer = tokenizer
        self.label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on', 'Ġat', 'Ġin', 'Ġof', 'Ġfor', 'Ġon']
        self.label_dict = {'**GW**':0, 'at':1, 'in':2, 'of':3, 'for':4, 'on':5,
                           'Ġat':1, 'Ġin':2, 'Ġof':3, 'Ġfor':4, 'Ġon':5}
        self.label_stat = [0,0,0,0,0,0]
        self.labels = labels
        self.label_num = 0

    def tokenizerFunc(self,str):
        x = self.tokenizer.encode(str)
        return {"input_ids":x.ids, "tokens":x.tokens, "attention_mask":x.attention_mask}

    def maskFunc(self, str):
        data_dict = self.tokenizerFunc(str)
        n = len(data_dict["input_ids"])
        data_dict["label"] = [-100]*n
        for i in range(n):
            if data_dict["tokens"][i] in self.label_list:
                data_dict["attention_mask"][i] = 0
                data_dict["label"][i] = self.label_dict[data_dict["tokens"][i]]
                # list the labels
                if (data_dict["tokens"][i] == '**GW**'):
                    data_dict["label"][i] = self.label_dict[self.labels[self.label_num]]
                    self.label_num += 1
        # padding
        data_dict['input_ids'] += [0]*(128-n)
        data_dict['tokens'] += ['']*(128-n)
        data_dict['attention_mask'] += [0]*(128-n)
        data_dict['label'] += [-100]*(128-n)
        return data_dict

    def __call__(self, example):
        return self.maskFunc(example)


In [ ]:
dataGenerator = DataGenerator_valid(tokenizer, valid_data_label)
dict_data={'input_ids':[], 'tokens':[], 'attention_mask':[], 'label':[]}
for i in valid_data:
    example = dataGenerator(i)
    dict_data['input_ids'].append(example['input_ids'])
    dict_data['tokens'].append(example['tokens'])
    dict_data['attention_mask'].append(example['attention_mask'])
    dict_data['label'].append(example['label'])
print(dataGenerator.label_num)
from datasets import Dataset
valid_dataset = Dataset.from_dict(dict_data)

14634


## 5. Generate pertrained AutoTokenizer for pertrained model

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "distilbert-base-uncased"
tokenizer_pertrained = AutoTokenizer.from_pretrained(model_checkpoint)

https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmp7yb8tqw4


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json in cache at /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
creating metadata file for /root/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7ff7eb7f73eecdbb.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt in cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
creating metadata file for /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json not found in cache or force_download set to True, downloading to /root/.cache/huggingface/transformers/tmpj7nwbdw8


Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

storing https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json in cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
creating metadata file for /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /root/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading

In [ ]:
class DataGenerator_pertrained(object):
    '''使用pandas来完成数据提取（复杂度有点感人，但只能选择相信pandas内置函数的速度了）'''
    def __init__(self, tokenizer):
        # 分词器
        self.tokenizer = tokenizer
        self.label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on']
        self.label_dict = {'**GW**':0, 'at':1, 'in':2, 'of':3, 'for':4, 'on':5}

    def tokenizerFunc(self,str):
        x = self.tokenizer(str)
        return {"input_ids":x.input_ids, "attention_mask":x.attention_mask,
                "tokens":self.tokenizer.convert_ids_to_tokens(x["input_ids"])}

    def maskFunc(self, str):
        data_dict = self.tokenizerFunc(str)
        n = len(data_dict["input_ids"])
        data_dict["label"] = [-100]*n
        for i in range(n):
            if data_dict["tokens"][i] in self.label_list:
                data_dict["attention_mask"][i] = 0
                data_dict["label"][i] = self.label_dict[data_dict["tokens"][i]]
        data_dict['input_ids'] += [0]*(200-n)
        data_dict['attention_mask'] += [0]*(200-n)
        data_dict['tokens'] += ['']*(200-n)
        data_dict['label'] += [-100]*(200-n)
        return data_dict

    def __call__(self, example):
        return self.maskFunc(example)



In [ ]:
class DataGenerator_valid_pertrained(object):
    '''使用pandas来完成数据提取（复杂度有点感人，但只能选择相信pandas内置函数的速度了）'''
    def __init__(self, tokenizer, labels):
        # 分词器
        self.tokenizer = tokenizer
        self.label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on']
        self.label_dict = {'**GW**':0, 'at':1, 'in':2, 'of':3, 'for':4, 'on':5}
        self.labels = labels
        self.label_num = 0

    def tokenizerFunc(self,str):
        x = self.tokenizer(str)
        return {"input_ids":x.input_ids, "attention_mask":x.attention_mask,
                "tokens":self.tokenizer.convert_ids_to_tokens(x["input_ids"])}

    def maskFunc(self, str):
        data_dict = self.tokenizerFunc(str)
        n = len(data_dict["input_ids"])
        data_dict["label"] = [-100]*n
        for i in range(n):
            if data_dict["tokens"][i] in self.label_list:
                data_dict["attention_mask"][i] = 0
                data_dict["label"][i] = self.label_dict[data_dict["tokens"][i]]
                # list the labels
                if (data_dict["tokens"][i] == '**GW**'):
                    data_dict["label"][i] = self.label_dict[self.labels[self.label_num]]
                    self.label_num += 1
        # padding
        data_dict['input_ids'] += [0]*(128-n)
        data_dict['tokens'] += ['']*(128-n)
        data_dict['attention_mask'] += [0]*(128-n)
        data_dict['label'] += [-100]*(128-n)
        return data_dict

    def __call__(self, example):
        return self.maskFunc(example)


In [ ]:
dataGenerator = DataGenerator_pertrained(tokenizer_pertrained)
dict_data={'input_ids':[], 'tokens':[], 'attention_mask':[], 'labels':[]}
for i in train_data:
    example = dataGenerator(i)
    dict_data['input_ids'].append(example['input_ids'])
    dict_data['tokens'].append(example['tokens'])
    dict_data['attention_mask'].append(example['attention_mask'])
    dict_data['labels'].append(example['label'])

from datasets import Dataset
dataset = Dataset.from_dict(dict_data)

In [ ]:
len(dataset[0]['tokens'])

200

In [ ]:
train_dataset=dataset
dataGenerator = DataGenerator_valid_pertrained(tokenizer_pertrained, valid_data_label)
dict_data={'input_ids':[], 'tokens':[], 'attention_mask':[], 'labels':[]}
for i in valid_data:
    example = dataGenerator(i)
    dict_data['input_ids'].append(example['input_ids'])
    dict_data['tokens'].append(example['tokens'])
    dict_data['attention_mask'].append(example['attention_mask'])
    dict_data['labels'].append(example['label'])
print(dataGenerator.label_num)
from datasets import Dataset
valid_dataset = Dataset.from_dict(dict_data)

0


In [ ]:
len(valid_data_label)

14634

In [ ]:
import datasets

dataset.shuffle()
sub_dataset = [dataset.shard(num_shards =10 , index = i) for i in range(10)]
train_set = datasets.concatenate_datasets([sub_dataset[j] for j in range(10) if j!=0])
text_set = sub_dataset[0]

## 6. Evaluate the output
我们要做的是一个五分类Token Classification（六分类？可能有样本不均衡的问题），要求是对每个位置mask然后推断这个位置可能是什么介词。
* 方案一：直接对整个句子做分类
* 方案二：对位置的上下文信息做分类


In [ ]:
!pip install seqeval
import numpy as np
from datasets import load_metric

metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[int(p)] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[int(l)] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

     |████████████████████████████████| 43 kB 1.2 MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16181 sha256=42b3cb4870afaef361e55f1fdeb5078907f3596713197b86aa342b980def2d78
  Stored in directory: /root/.cache/pip/wheels/05/96/ee/7cac4e74f3b19e3158dce26a20a1c86b3533c43ec72a549fd7
Successfully built seqeval


Downloading:   0%|          | 0.00/2.48k [00:00<?, ?B/s]

# 2. A Baseline by PerTrained Bert in Huggingface
**this part should not be excuted for final result.**

https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=imY1oC3SIrJf

The Motivition of this part is to find the An estimate of the theoretical best value.

The function of pertraining is in this paper:

https://openaccess.thecvf.com/content_ICCV_2019/papers/He_Rethinking_ImageNet_Pre-Training_ICCV_2019_paper.pdf



In [ ]:
import numpy as np
!pip install seqeval
from datasets import load_metric

metric = load_metric("seqeval")
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    print(true_predictions)
    print(true_labels)

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on']
model_checkpoint = "distilbert-base-uncased"
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"test-3361-pertrain",
    evaluation_strategy = "steps",
    learning_rate=2e-4,# TO steps
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=6,
    weight_decay=0.0001,# TO change
)

from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
del model
model = AutoModelForTokenClassification.from_pretrained(model_checkpoint, num_labels=len(label_list))

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "lab

In [ ]:
import torch
import time
import torch.nn as nn
import torch.nn.functional as F
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        def get_one_hot(label):
            size = label.size()
            # create one-hot vector for label map
            oneHot_size = (size[0], size[1])
            input_label = torch.FloatTensor(torch.Size(oneHot_size)).zero_().cuda()
            input_label = input_label.cuda().scatter_(1, label.data.long(), 1.0)
            return input_label
        labels = inputs.pop("labels")
        
        labels = labels.unsqueeze(2).repeat(1,1,6)

        outputs = model(**inputs)
        logits = outputs.logits
        # logits = F.softmax(logits, dim=2)


        logits = logits[labels!=-100]
        labels = labels[labels!=-100]

        logits = logits.view(-1, self.model.config.num_labels)
        labels = get_one_hot(labels.view(-1, self.model.config.num_labels))

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits,
                        labels.float())
        return (loss, outputs) if return_outputs else loss

In [ ]:
del trainer
trainer = MultilabelTrainer(
    model,
    args,
    train_dataset=train_set,
    eval_dataset=text_set,
    compute_metrics=compute_metrics,
    
)


In [ ]:
trainer.args=args

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens.
***** Running training *****
  Num examples = 60145
  Num Epochs = 6
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 22560


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
500,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
1000,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000
1500,0.000000,0.000000,1.000000,1.000000,1.000000,1.000000


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: tokens.
***** Running Evaluation *****
  Num examples = 6683
  Batch size = 16


[['of'], ['of', 'in'], ['on'], ['at', 'in'], ['for', 'for', 'for'], ['of', 'of'], ['for'], ['at'], ['of', 'for'], ['for', 'on', 'for'], ['of', 'of', 'in', 'in'], ['in', 'of', 'in', 'of'], ['in', 'on', 'of', 'of', 'of'], ['at', 'in'], ['of', 'in', 'of'], ['of', 'for'], ['for', 'in', 'of'], ['of', 'of'], ['of'], ['for', 'in'], ['of'], ['in', 'of', 'of'], ['in', 'of'], ['of', 'in', 'in', 'of'], ['for'], ['in'], ['in', 'in', 'of', 'of', 'of'], ['in', 'in', 'for'], ['in', 'in'], ['in', 'of'], ['of', 'of'], ['for', 'for', 'of', 'in'], ['for', 'of'], ['of'], ['of'], ['in'], ['in', 'of', 'for'], ['in'], ['in', 'on', 'on'], ['of'], ['of'], ['of'], ['in'], ['in', 'of'], ['of', 'of'], ['in', 'for', 'in'], ['in', 'in', 'for'], ['for', 'in'], ['in', 'in'], ['in'], ['at'], ['for'], ['of'], ['on', 'for', 'of'], ['of'], ['of'], ['in', 'for'], ['in', 'of', 'for'], ['on'], ['at', 'of', 'for'], ['at', 'in'], ['of'], ['in'], ['at'], ['on'], ['of'], ['on', 'at', 'at'], ['in', 'of', 'in'], ['of'], ['on', 'o

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: of seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: in seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: on seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: at seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: for seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Saving model checkpoint to test-3361-pertrain/checkpoint-500
Configuration saved in test-3361-

[['of'], ['of', 'in'], ['on'], ['at', 'in'], ['for', 'for', 'for'], ['of', 'of'], ['for'], ['at'], ['of', 'for'], ['for', 'on', 'for'], ['of', 'of', 'in', 'in'], ['in', 'of', 'in', 'of'], ['in', 'on', 'of', 'of', 'of'], ['at', 'in'], ['of', 'in', 'of'], ['of', 'for'], ['for', 'in', 'of'], ['of', 'of'], ['of'], ['for', 'in'], ['of'], ['in', 'of', 'of'], ['in', 'of'], ['of', 'in', 'in', 'of'], ['for'], ['in'], ['in', 'in', 'of', 'of', 'of'], ['in', 'in', 'for'], ['in', 'in'], ['in', 'of'], ['of', 'of'], ['for', 'for', 'of', 'in'], ['for', 'of'], ['of'], ['of'], ['in'], ['in', 'of', 'for'], ['in'], ['in', 'on', 'on'], ['of'], ['of'], ['of'], ['in'], ['in', 'of'], ['of', 'of'], ['in', 'for', 'in'], ['in', 'in', 'for'], ['for', 'in'], ['in', 'in'], ['in'], ['at'], ['for'], ['of'], ['on', 'for', 'of'], ['of'], ['of'], ['in', 'for'], ['in', 'of', 'for'], ['on'], ['at', 'of', 'for'], ['at', 'in'], ['of'], ['in'], ['at'], ['on'], ['of'], ['on', 'at', 'at'], ['in', 'of', 'in'], ['of'], ['on', 'o

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: of seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: in seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: on seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: at seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: for seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Saving model checkpoint to test-3361-pertrain/checkpoint-1000
Configuration saved in test-3361

[['of'], ['of', 'in'], ['on'], ['at', 'in'], ['for', 'for', 'for'], ['of', 'of'], ['for'], ['at'], ['of', 'for'], ['for', 'on', 'for'], ['of', 'of', 'in', 'in'], ['in', 'of', 'in', 'of'], ['in', 'on', 'of', 'of', 'of'], ['at', 'in'], ['of', 'in', 'of'], ['of', 'for'], ['for', 'in', 'of'], ['of', 'of'], ['of'], ['for', 'in'], ['of'], ['in', 'of', 'of'], ['in', 'of'], ['of', 'in', 'in', 'of'], ['for'], ['in'], ['in', 'in', 'of', 'of', 'of'], ['in', 'in', 'for'], ['in', 'in'], ['in', 'of'], ['of', 'of'], ['for', 'for', 'of', 'in'], ['for', 'of'], ['of'], ['of'], ['in'], ['in', 'of', 'for'], ['in'], ['in', 'on', 'on'], ['of'], ['of'], ['of'], ['in'], ['in', 'of'], ['of', 'of'], ['in', 'for', 'in'], ['in', 'in', 'for'], ['for', 'in'], ['in', 'in'], ['in'], ['at'], ['for'], ['of'], ['on', 'for', 'of'], ['of'], ['of'], ['in', 'for'], ['in', 'of', 'for'], ['on'], ['at', 'of', 'for'], ['at', 'in'], ['of'], ['in'], ['at'], ['on'], ['of'], ['on', 'at', 'at'], ['in', 'of', 'in'], ['of'], ['on', 'o

/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: of seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: in seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: on seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: at seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
/usr/local/lib/python3.7/dist-packages/seqeval/metrics/sequence_labeling.py:171: UserWarning: for seems not to be NE tag.
  warnings.warn('{} seems not to be NE tag.'.format(chunk))
Saving model checkpoint to test-3361-pertrain/checkpoint-1500
Configuration saved in test-3361

KeyboardInterrupt: ignored

In [ ]:
trainer.eval_dataset

Dataset({
    features: ['input_ids', 'tokens', 'attention_mask', 'labels'],
    num_rows: 7057
})

# 3. Pertrain model by masklm task

In [ ]:
dataset

Dataset({
    features: ['input_ids'],
    num_rows: 66828
})

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("/content/drive/MyDrive/COMP3361/model20", max_len=512)

Didn't find file /content/drive/MyDrive/COMP3361/model20/tokenizer.json. We won't load it.
Didn't find file /content/drive/MyDrive/COMP3361/model20/added_tokens.json. We won't load it.
Didn't find file /content/drive/MyDrive/COMP3361/model20/special_tokens_map.json. We won't load it.
Didn't find file /content/drive/MyDrive/COMP3361/model20/tokenizer_config.json. We won't load it.
loading file /content/drive/MyDrive/COMP3361/model20/vocab.json
loading file /content/drive/MyDrive/COMP3361/model20/merges.txt
loading file None
loading file None
loading file None
loading file None
loading configuration file /content/drive/MyDrive/COMP3361/model20/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "inte

In [ ]:
model=model.from_pretrained("/content/drive/MyDrive/COMP3361/model20")

loading configuration file /content/drive/MyDrive/COMP3361/model20/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.11.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 10000
}

loading weights file /content/drive/MyDrive/COMP3361/model20/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForMaskedLM.

All the weights of RobertaForMaskedLM were initialized from 

In [ ]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=10000,
    max_position_embeddings=130,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaForMaskedLM
model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()

50911504

In [ ]:
dataset

Dataset({
    features: ['input_ids'],
    num_rows: 66828
})

In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/COMP3361",
    learning_rate = 5e-4,
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_gpu_train_batch_size=64,
    save_steps=10000,
    prediction_loss_only=True,
    warmup_steps = 10000,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
trainer.train()

Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.
***** Running training *****
  Num examples = 66828
  Num Epochs = 100
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 104500
Using deprecated `--per_gpu_train_batch_size` argument which will be removed in a future version. Using `--per_device_train_batch_size` is preferred.


Step,Training Loss
500,7.692900
1000,6.713400
1500,6.409400
2000,6.136900
2500,5.779500
3000,5.395300
3500,5.118500
4000,4.916600
4500,4.757400
5000,4.620900


Saving model checkpoint to /content/drive/MyDrive/COMP3361/checkpoint-10000
Configuration saved in /content/drive/MyDrive/COMP3361/checkpoint-10000/config.json
Model weights saved in /content/drive/MyDrive/COMP3361/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/COMP3361/checkpoint-20000
Configuration saved in /content/drive/MyDrive/COMP3361/checkpoint-20000/config.json
Model weights saved in /content/drive/MyDrive/COMP3361/checkpoint-20000/pytorch_model.bin
Saving model checkpoint to /content/drive/MyDrive/COMP3361/checkpoint-30000
Configuration saved in /content/drive/MyDrive/COMP3361/checkpoint-30000/config.json
Model weights saved in /content/drive/MyDrive/COMP3361/checkpoint-30000/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/COMP3361/checkpoint-10000] due to args.save_total_limit


Step,Training Loss
500,7.692900
1000,6.713400
1500,6.409400
2000,6.136900
2500,5.779500
3000,5.395300
3500,5.118500
4000,4.916600
4500,4.757400
5000,4.620900


Saving model checkpoint to /content/drive/MyDrive/COMP3361/checkpoint-40000
Configuration saved in /content/drive/MyDrive/COMP3361/checkpoint-40000/config.json
Model weights saved in /content/drive/MyDrive/COMP3361/checkpoint-40000/pytorch_model.bin
Deleting older checkpoint [/content/drive/MyDrive/COMP3361/checkpoint-20000] due to args.save_total_limit


KeyboardInterrupt: ignored

In [ ]:
trainer.save_model("/content/drive/MyDrive/COMP3361/model_pertrain")

Saving model checkpoint to /content/drive/MyDrive/COMP3361/model_pertrain
Configuration saved in /content/drive/MyDrive/COMP3361/model_pertrain/config.json
Model weights saved in /content/drive/MyDrive/COMP3361/model_pertrain/pytorch_model.bin


In [ ]:
tokenizer.save_model("/content/drive/MyDrive/COMP3361/model_pertrain")

['/content/drive/MyDrive/COMP3361/model_pertrain/vocab.json',
 '/content/drive/MyDrive/COMP3361/model_pertrain/merges.txt']

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="/content/drive/MyDrive/COMP3361/checkpoint-10000",
    tokenizer="/content/drive/MyDrive/COMP3361/model_pertrain",

)

loading configuration file /content/drive/MyDrive/COMP3361/checkpoint-10000/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.11.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 10000
}

loading configuration file /content/drive/MyDrive/COMP3361/checkpoint-10000/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
 

In [ ]:
valid_data[4]

'homarus gammarus is a large <unk> , with a body length up to 60 centimetres ( 24 **GW** ) and weighing up to 5 – 6 kilograms ( 11 – 13 lb ) , although the lobsters caught **GW** lobster pots are usually 23 – 38 cm ( 9 – 15 **GW** ) long and weigh 0 @'

In [ ]:
valid_data[0:2]

['homarus gammarus , known as the european lobster or common lobster , is a species **GW** <unk> lobster from the eastern atlantic ocean , mediterranean sea and parts **GW** the black sea ',
 ' it may grow to a length **GW** 60 cm ( 24 **GW** ) and a mass **GW** 6 kilograms ( 13 lb ) , and bears a conspicuous pair **GW** claws ']

In [ ]:
valid_data_label[0:10]

['of', 'of', 'of', 'in', 'of', 'of', 'in', 'on', 'in', 'for']

In [ ]:
fill_mask("the pair married <mask> may 2004")

[{'score': 0.07565198838710785,
  'sequence': 'the pair married " may 2004',
  'token': 310,
  'token_str': ' "'},
 {'score': 0.0420541986823082,
  'sequence': 'the pair married in may 2004',
  'token': 287,
  'token_str': ' in'},
 {'score': 0.038955457508563995,
  'sequence': 'the pair married the may 2004',
  'token': 266,
  'token_str': ' the'},
 {'score': 0.03552432358264923,
  'sequence': 'the pair married it may 2004',
  'token': 357,
  'token_str': ' it'},
 {'score': 0.0342252179980278,
  'sequence': 'the pair married ] may 2004',
  'token': 1312,
  'token_str': ' ]'}]

In [ ]:
def generate_data(st):
    li=[]
    while st.find("**GW**")!=-1:
        s = st[:]
        s=s.replace("**GW**", "<mask>", 1)
        s=s.replace("**GW**", "<unk>")
        li.append(s)
        st=st.replace("**GW**", "<unk>", 1)
    return li

In [ ]:
generate_data(" is a species **GW** <unk> lobster from the eastern atlantic ocean , mediterranean sea and parts **GW** the black sea ")

[' is a species <mask> <unk> lobster from the eastern atlantic ocean , mediterranean sea and parts <unk> the black sea ',
 ' is a species <unk> <unk> lobster from the eastern atlantic ocean , mediterranean sea and parts <mask> the black sea ']

In [ ]:
valid_data_label[1]

'of'

In [ ]:
confution_matrix=np.zeros([6,6])

In [ ]:
confution_matrix

array([[0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0.]])

In [ ]:
label_num = 0
label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on', 'Ġat', 'Ġin',
              'Ġof', 'Ġfor', 'Ġon',' at', ' in', ' of', ' for', ' on']
label_dict = {'**GW**':0, 'at':1, 'in':2, 'of':3, 'for':4, 'on':5,
                           'Ġat':1, 'Ġin':2, 'Ġof':3, 'Ġfor':4, 'Ġon':5,
                           ' at':1, ' in':2, ' of':3, ' for':4, ' on':5,
                }
preds=[]
pred_str=[]
acc_pred = 0
for i in valid_data:
    for s in generate_data(i):
        predict_list=fill_mask(s)
        pred_str.append(s)
        flag = False
        for ele in predict_list:
            if (ele["token_str"] in label_list):
                if (not flag and label_dict[valid_data_label[label_num]]
                        ==label_dict[ele["token_str"]]):
                    acc_pred += 1
                flag = True
                confution_matrix[label_dict[valid_data_label[label_num]]][label_dict[ele["token_str"]]] += 1
                preds.append(label_dict[ele["token_str"]])
                break
        if (not flag and label_dict[valid_data_label[label_num]]==3):
            acc_pred += 1
            confution_matrix[label_dict[valid_data_label[label_num]]][0] += 1
        if (not flag):
            preds.append(0)
        label_num += 1
        if label_num % 100==0:
            print(label_num)
        if label_num==20000:
            break
    if label_num==20000:
        break

print(acc_pred/label_num)

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
0.6381030476971437


In [ ]:
label_num = 0
label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on', 'Ġat', 'Ġin',
              'Ġof', 'Ġfor', 'Ġon',' at', ' in', ' of', ' for', ' on']
label_dict = {'**GW**':0, 'at':1, 'in':2, 'of':3, 'for':4, 'on':5,
                           'Ġat':1, 'Ġin':2, 'Ġof':3, 'Ġfor':4, 'Ġon':5,
                           ' at':1, ' in':2, ' of':3, ' for':4, ' on':5,
                }
preds=[]
pred_str=[]
acc_pred = 0
for i in valid_data:
    for s in generate_data(i):
        predict_list=fill_mask(s)
        pred_str.append(s)
        flag = False
        for ele in predict_list:
            if (ele["token_str"] in label_list):
                if (not flag and label_dict[valid_data_label[label_num]]
                        ==label_dict[ele["token_str"]]):
                    acc_pred += 1
                flag = True
                confution_matrix[label_dict[valid_data_label[label_num]]][label_dict[ele["token_str"]]] += 1
                preds.append(label_dict[ele["token_str"]])
                break
        if (not flag and label_dict[valid_data_label[label_num]]==3):
            acc_pred += 1
            confution_matrix[label_dict[valid_data_label[label_num]]][0] += 1
        if (not flag):
            preds.append(0)
        label_num += 1
        if label_num % 1000==0:
            print(label_num)
        if label_num==20000:
            break
    if label_num==20000:
        break

print(acc_pred/label_num)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
0.6118627852945197


In [ ]:
label_num = 0
label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on', 'Ġat', 'Ġin',
              'Ġof', 'Ġfor', 'Ġon',' at', ' in', ' of', ' for', ' on']
label_dict = {'**GW**':0, 'at':1, 'in':2, 'of':3, 'for':4, 'on':5,
                           'Ġat':1, 'Ġin':2, 'Ġof':3, 'Ġfor':4, 'Ġon':5,
                           ' at':1, ' in':2, ' of':3, ' for':4, ' on':5,
                }
preds=[]
pred_str=[]
acc_pred = 0
for i in valid_data:
    for s in generate_data(i):
        predict_list=fill_mask(s)
        pred_str.append(s)
        flag = False
        for ele in predict_list:
            if (ele["token_str"] in label_list):
                if (not flag and label_dict[valid_data_label[label_num]]
                        ==label_dict[ele["token_str"]]):
                    acc_pred += 1
                flag = True
                confution_matrix[label_dict[valid_data_label[label_num]]][label_dict[ele["token_str"]]] += 1
                preds.append(label_dict[ele["token_str"]])
                break
        if (not flag and label_dict[valid_data_label[label_num]]==3):
            acc_pred += 1
            confution_matrix[label_dict[valid_data_label[label_num]]][0] += 1
        if (not flag):
            preds.append(0)
        label_num += 1
        if label_num % 1000==0:
            print(label_num)
        if label_num==20000:
            break
    if label_num==20000:
        break

print(acc_pred/label_num)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
0.6076260762607626


In [ ]:
 'at', 'in', 'of', 'for', 'on
confution_matrix

array([[   0.,    0.,    0.,    0.,    0.,    0.],
       [   0.,  241.,  233.,  379.,   25.,   15.],
       [   0.,   42., 2677., 1540.,   80.,   18.],
       [ 201.,   16.,  196., 5457.,   40.,   16.],
       [   0.,   28.,  258.,  606.,  214.,   11.],
       [   0.,   46.,  381.,  514.,   44.,  548.]])

In [ ]:
preds_new=[]
for i in preds_lists:
    flag = False
    for ele in i:
        if (ele["token_str"] in label_list):
            flag = True
            preds_new.append(label_dict[ele["token_str"]])
            break
    if (not flag):
        preds_new.append(0)

In [ ]:
len(valid_data_label)

14634

In [ ]:
preds_new[10000:10010]

[3, 1, 2, 3, 2, 2, 2, 1, 2, 0]

In [ ]:
preds[10000:10010]

[3, 5, 5, 1, 5, 5, 3, 3, 3, 3]

In [ ]:
[label_dict[valid_data_label[i]]for i in range(10000,10010)]

[3, 5, 2, 3, 2, 1, 2, 1, 2, 2]

In [ ]:
pred_str[10001]

' the mill was then supplied with lights and <unk> <mask> each floor '

In [ ]:
fill_mask(" the mill was then supplied with lights and <unk> <mask> each floor ")

[{'score': 0.19861899316310883,
  'sequence': ' the mill was then supplied with lights and the each floor ',
  'token': 266,
  'token_str': ' the'},
 {'score': 0.15827299654483795,
  'sequence': ' the mill was then supplied with lights and in each floor ',
  'token': 287,
  'token_str': ' in'},
 {'score': 0.10238978266716003,
  'sequence': ' the mill was then supplied with lights and on each floor ',
  'token': 327,
  'token_str': ' on'},
 {'score': 0.08095929771661758,
  'sequence': ' the mill was then supplied with lights and from each floor ',
  'token': 393,
  'token_str': ' from'},
 {'score': 0.05463002249598503,
  'sequence': ' the mill was then supplied with lights andthe each floor ',
  'token': 491,
  'token_str': 'the'}]

In [ ]:
preds_lists[10001]

[{'score': 0.17958012223243713,
  'sequence': ' the mill was then supplied with lights and  at each floor ',
  'token': 371,
  'token_str': ' at'},
 {'score': 0.16158427298069,
  'sequence': ' the mill was then supplied with lights and  on each floor ',
  'token': 327,
  'token_str': ' on'},
 {'score': 0.09536627680063248,
  'sequence': ' the mill was then supplied with lights and  in each floor ',
  'token': 287,
  'token_str': ' in'},
 {'score': 0.07280541956424713,
  'sequence': ' the mill was then supplied with lights and  off each floor ',
  'token': 593,
  'token_str': ' off'},
 {'score': 0.044122349470853806,
  'sequence': ' the mill was then supplied with lights and , each floor ',
  'token': 270,
  'token_str': ','}]

In [ ]:
fill_mask(pred_str[10003])

[{'score': 0.7821491360664368,
  'sequence': ' a 21 horsepower ( 16 kw )  &   diesel engine had been purchased  1981 and was fitted onto a permanent concrete base with the aid of a mobile crane ',
  'token': 294,
  'token_str': ' of'},
 {'score': 0.06610605120658875,
  'sequence': ' a 21 horsepower ( 16 kw )  &   diesel engine had been purchased  1981 and was fitted onto a permanent concrete base with the aid @-@ a mobile crane ',
  'token': 339,
  'token_str': ' @-@'},
 {'score': 0.009596086107194424,
  'sequence': ' a 21 horsepower ( 16 kw )  &   diesel engine had been purchased  1981 and was fitted onto a permanent concrete base with the aid, a mobile crane ',
  'token': 270,
  'token_str': ','},
 {'score': 0.007852841168642044,
  'sequence': ' a 21 horsepower ( 16 kw )  &   diesel engine had been purchased  1981 and was fitted onto a permanent concrete base with the aid using a mobile crane ',
  'token': 1851,
  'token_str': ' using'},
 {'score': 0.005792343057692051,
  'sequence':

In [ ]:
pred_list=fill_mask(" a 21 horsepower ( 16 kw ) <unk> & <unk> <unk> diesel engine had been purchased <mask> 1981 and was fitted onto a permanent concrete base with the aid <unk> a mobile crane ")

[{'score': 0.6397501230239868,
  'sequence': ' a 21 horsepower ( 16 kw )  &   diesel engine had been purchased in 1981 and was fitted onto a permanent concrete base with the aid  a mobile crane ',
  'token': 287,
  'token_str': ' in'},
 {'score': 0.08526991307735443,
  'sequence': ' a 21 horsepower ( 16 kw )  &   diesel engine had been purchased by 1981 and was fitted onto a permanent concrete base with the aid  a mobile crane ',
  'token': 367,
  'token_str': ' by'},
 {'score': 0.02322770282626152,
  'sequence': ' a 21 horsepower ( 16 kw )  &   diesel engine had been purchased the 1981 and was fitted onto a permanent concrete base with the aid  a mobile crane ',
  'token': 266,
  'token_str': ' the'},
 {'score': 0.020030885934829712,
  'sequence': ' a 21 horsepower ( 16 kw )  &   diesel engine had been purchased for 1981 and was fitted onto a permanent concrete base with the aid  a mobile crane ',
  'token': 334,
  'token_str': ' for'},
 {'score': 0.0173040684312582,
  'sequence': ' a

In [ ]:
for ele in predict_list:
    if (ele["token_str"] in label_list):
        print(ele["token_str"])

 in


In [ ]:
label_num

14634

In [ ]:
model.from_pretrained("/content/drive/MyDrive/COMP3361/model")

loading configuration file /content/drive/MyDrive/COMP3361/model/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.11.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 10000
}

loading weights file /content/drive/MyDrive/COMP3361/model/pytorch_model.bin
All model checkpoint weights were used when initializing RobertaForMaskedLM.

All the weights of RobertaForMaskedLM were initialized from the 

RobertaForMaskedLM(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(10000, 768, padding_idx=1)
      (position_embeddings): Embedding(130, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNor

# 3.1 Argument the data by pertrained model
把数据量增加10倍防止过拟合(?

# 4. A Baseline by Huggingface-Transformers
Mainly refer to this:

https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=G-kkz81OY6xH

and:

https://colab.research.google.com/github/huggingface/notebooks/blob/master/examples/token_classification.ipynb#scrollTo=imY1oC3SIrJf

In [ ]:
import torch.nn as nn
from transformers import AutoModel 
class myModel(nn.Module):
    def __init__(self, freeze_bert=False, model_name="/content/drive/MyDrive/COMP3361/checkpoint-30000", hidden_size=768, num_classes=6):
        super(myModel, self).__init__()
        # 返回输出的隐藏层
        self.bert = AutoModel.from_pretrained(model_name, output_hidden_states=True, return_dict=True)
        for p in self.bert.parameters():
            p.requires_grad = False
        self.config = RobertaConfig(
            vocab_size=10000,
            max_position_embeddings=130,
            num_attention_heads=12,
            num_hidden_layers=6,
            type_vocab_size=1,
            num_labels=6
        )
        for p in self.bert.parameters():
            p.requires_grad = False
        '''
        self.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(hidden_size*4, 2048),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(2048, 1024),
            nn.ReLU(True),
            nn.Linear(1024, num_classes),
        )
        '''
        self.fc = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(hidden_size*4, num_classes, bias=False),
        )
        self.hidden_size=hidden_size
  
    def forward(self, input_ids, attention_mask):
    	#其实只需要input_ids也可以
        outputs = self.bert(input_ids, attention_mask=attention_mask)
        # 将最后输入的隐藏层后四个进行拼接
        size=input_ids.shape
        hidden_states = torch.cat(tuple([outputs.hidden_states[i] for i in [-1, -2, -3, -4]]), dim=-1) # [bs, seq_len, hidden_dim*4]
        #first_hidden_states = hidden_states[:, 0, :] # [bs, hidden_dim*4] # 这里取的是cls的embedding表示，
        #它代表了一句话的embedding
        hidden_states = torch.reshape(hidden_states, [size[0]*size[1],self.hidden_size*4])
        
        logits = self.fc(hidden_states)
        return torch.reshape(logits, [size[0],size[1],6])

In [ ]:
del Mmodel
Mmodel = myModel()

loading configuration file /content/drive/MyDrive/COMP3361/checkpoint-30000/config.json
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 130,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "output_hidden_states": true,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.11.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 10000
}

loading weights file /content/drive/MyDrive/COMP3361/checkpoint-30000/pytorch_model.bin
Some weights of the model checkpoint at /content/drive/MyDrive/COMP3361/checkpoint-30000

In [ ]:
train_dataset

Dataset({
    features: ['input_ids', 'tokens', 'attention_mask', 'label'],
    num_rows: 66828
})

In [ ]:
model.bert.num_parameters()

50899968

In [ ]:
import torch
import time
import torch.nn as nn
import torch.nn.functional as F
class MultilabelTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        def get_one_hot(label):
            size = label.size()
            # create one-hot vector for label map
            oneHot_size = (size[0], size[1])
            input_label = torch.FloatTensor(torch.Size(oneHot_size)).zero_().cuda()
            input_label = input_label.cuda().scatter_(1, label.data.long(), 1.0)
            return input_label
        labels = inputs.pop("labels")
        
        labels = labels.unsqueeze(2).repeat(1,1,6)

        outputs = model(**inputs)
        logits = outputs
        # logits = F.softmax(logits, dim=2)
        
        logits = logits[labels!=-100]
        labels = labels[labels!=-100]

        logits = logits.view(-1, self.model.config.num_labels)
        labels = get_one_hot(labels.view(-1, self.model.config.num_labels))

        loss_fct = nn.BCEWithLogitsLoss()
        loss = loss_fct(logits,
                        labels.float())
        return (loss, {"logics":outputs}) if return_outputs else loss

In [ ]:
from transformers import TrainingArguments, Trainer

label_list = ['**GW**', 'at', 'in', 'of', 'for', 'on']
model_checkpoint = "distilbert-base-uncased"
model_name = model_checkpoint.split("/")[-1]
args = TrainingArguments(
    f"test-3361-pertrain",
    evaluation_strategy = "epoch",
    learning_rate=5e-4,# TO steps
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps = 10000,
    num_train_epochs=5,
    weight_decay=0.01,# TO change
)


PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
del trainer
trainer = MultilabelTrainer(
    Mmodel,
    args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    compute_metrics=compute_metrics,
    
)


In [ ]:
print(train_dataset[0].keys())

dict_keys(['input_ids', 'tokens', 'attention_mask', 'labels'])


In [ ]:
valid_dataset

Dataset({
    features: ['input_ids', 'tokens', 'attention_mask', 'label'],
    num_rows: 7057
})

In [ ]:
valid_data_label[0]

'of'

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `myModel.forward` and have been ignored: tokens.
***** Running training *****
  Num examples = 66828
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20885


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.117300,0.383178,0.427636,0.379593,0.402185,0.514629
2,0.092600,0.426490,0.434701,0.397103,0.415052,0.525567
3,0.087800,0.467351,0.419564,0.386057,0.402114,0.510468
4,0.088700,0.483078,0.414684,0.379429,0.396274,0.509864
5,0.084900,0.489175,0.416993,0.374274,0.394480,0.509261


Saving model checkpoint to test-3361-pertrain/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-3500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-4000
Trainer.model i

TrainOutput(global_step=20885, training_loss=0.11652383126223918, metrics={'train_runtime': 969.2224, 'train_samples_per_second': 344.751, 'train_steps_per_second': 21.548, 'total_flos': 0.0, 'train_loss': 0.11652383126223918, 'epoch': 5.0})

In [ ]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `myModel.forward` and have been ignored: tokens.
***** Running training *****
  Num examples = 66828
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 20885


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.271100,0.324210,0.463434,0.383193,0.419511,0.539659
2,0.177700,0.321954,0.466642,0.409786,0.436370,0.552879
3,0.146200,0.332010,0.459957,0.415432,0.436562,0.547175
4,0.138200,0.338592,0.454390,0.412896,0.432650,0.542545
5,0.133800,0.341419,0.452823,0.410768,0.430772,0.539592


Saving model checkpoint to test-3361-pertrain/checkpoint-500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-1000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-1500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-2000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-2500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-3000
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-3500
Trainer.model is not a `PreTrainedModel`, only saving its state dict.
Saving model checkpoint to test-3361-pertrain/checkpoint-4000
Trainer.model i

TrainOutput(global_step=20885, training_loss=0.2041428994984041, metrics={'train_runtime': 967.5521, 'train_samples_per_second': 345.346, 'train_steps_per_second': 21.585, 'total_flos': 0.0, 'train_loss': 0.2041428994984041, 'epoch': 5.0})

In [ ]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `RobertaForMaskedLM.forward` and have been ignored: tokens.
***** Running Evaluation *****
  Num examples = 7057
  Batch size = 16


tensor([[[ -7.5323, -11.4799,  -8.0604,  ...,  -4.3756,  -3.9447,  -4.1996],
         [ -7.6627, -11.1700,  -8.3156,  ...,  -4.2197,  -4.3373,  -3.9465],
         [ -7.4582, -11.3217,  -7.9900,  ...,  -4.3797,  -4.0419,  -4.4385],
         ...,
         [ -7.8156, -10.5630,  -9.0040,  ...,  -4.7291,  -4.1973,  -3.4194],
         [ -7.6698, -11.4350,  -8.4166,  ...,  -4.2917,  -3.9871,  -4.1384],
         [ -7.6827, -11.4567,  -8.3173,  ...,  -4.4368,  -3.8642,  -4.1226]],

        [[ -6.8641, -13.4432,  -6.5482,  ...,   0.6327,  -0.5739,  -7.3641],
         [ -6.9047, -13.5174,  -6.3943,  ...,   0.5885,  -0.9123,  -7.4087],
         [ -7.2911, -13.6999,  -6.2835,  ...,   0.1927,  -3.8742,  -8.5399],
         ...,
         [ -7.3463, -13.7677,  -6.3289,  ...,  -0.0338,  -4.2412,  -8.5319],
         [ -7.2218, -13.6749,  -6.2590,  ...,   0.3822,  -3.4484,  -8.5735],
         [ -6.9657, -13.4939,  -6.3864,  ...,   0.4367,  -0.9164,  -7.1501]],

        [[ -7.9686, -11.1955,  -8.1824,  ...

IndexError: ignored

In [ ]:
model.output.keys()

odict_keys(['last_hidden_state', 'pooler_output', 'hidden_states'])

In [ ]:
model.output['last_hidden_state'].shape

torch.Size([16, 128, 768])

In [ ]:
len(model.output['hidden_states'])

7

In [ ]:
model.output['hidden_states'][6].shape

torch.Size([16, 128, 768])

In [ ]:
def save_with_huggingface_format(model):
    model.save_pretrained("/content/drive/MyDrive/COMP3361/model_fintuned")
save_with_huggingface_format(model)

# 5. Encapsulate the model and give the result
